### Fetching Cryptocurrency Data

In [19]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
import pandas as pd
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [20]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
data = r.json()
data = data['Data']
crypto_df = pd.DataFrame.from_dict(data)
crypto_df = crypto_df.T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.000244,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFramefrom pathlib import Path
file_path = ("crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df

FileNotFoundError: [Errno 2] No such file or directory: 'crypto_data.csv'

### Data Preprocessing

In [23]:
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]

In [24]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]

In [25]:
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
...,...,...,...,...,...,...
RONCOIN,RON,N/A,True,N/A,NaN,NaN
RISE,EverRise,N/A,True,N/A,71618033988,-1
STARTA,Starta,N/A,True,N/A,NaN,NaN
STA,Defi STOA,N/A,True,N/A,NaN,NaN


In [26]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df["Algorithm"] != "N/A"]

In [27]:
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
...,...,...,...,...,...,...
APTCOIN,Aptcoin,Scrypt-n,True,PoW,9800000,42000000
PSILOC,Psilocybin,SHA-256D,True,PoW/PoS,NaN,NaN
PESOBIT,PesoBit,Scrypt,True,PoW/PoS,NaN,NaN
OROCOIN,OroCoin,Scrypt,True,PoW,NaN,NaN


In [28]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns='IsTrading')

In [29]:
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
365,365Coin,X11,PoW/PoS,0,-1
404,404Coin,Scrypt,PoW/PoS,0,-1
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
...,...,...,...,...,...
APTCOIN,Aptcoin,Scrypt-n,PoW,9800000,42000000
PSILOC,Psilocybin,SHA-256D,PoW/PoS,NaN,NaN
PESOBIT,PesoBit,Scrypt,PoW/PoS,NaN,NaN
OROCOIN,OroCoin,Scrypt,PoW,NaN,NaN


In [30]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]> 0]

In [31]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [32]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname_df = pd.DataFrame(data = crypto_df, columns = ['CoinName'])

In [33]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = 'CoinName')

In [34]:
# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])

In [41]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummies)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,41.999952,42
NSR,PoS,PoS,6175806877.9692,0
TRI,X13,PoW/PoS,191623.903871,0
CMTC,Scrypt,PoW,872830,0
CHAT,Scrypt,PoW/PoS,1000000000,-1
...,...,...,...,...
DOGE,Scrypt,PoW,133959856383.705246,-1
ETH,Ethash,PoW,120325525.499,-1
BANANA,BEP-20 Token,NaN,134906772.60915,-1
APTCOIN,Scrypt-n,PoW,9800000,42000000


### Reducing Dimensions Using PCA

In [36]:
# Use PCA to reduce dimensions to 3 principal components
pca_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"])

NameError: name 'crypto_pca' is not defined

In [37]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

NameError: name 'crypto_pca' is not defined

In [38]:
pca_df

NameError: name 'pca_df' is not defined

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
km = KMeans(n_clusters=4, random_state=0)
# Fit the model
km.fit(pca_df)
# Predict clusters
predictions = km.predict(pca_df)
# Creating a new DataFrame including predicted clusters and cryptocurrencies features
pca_df['class'] = km.labels_
pca_df.index = crypto_df.index

In [ ]:
clustered_df = pd.concat([crypto_df, pca_df,coins_name], axis=1,)

In [ ]:
clustered_df

### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
MM_SCALER = MinMaxScaler()
plot_data =MM_SCALER.fit_transform(clustered_df[["MaxSupply", "TotalCoinsMined"]])
plot_df = pd.DataFrame(plot_data, columns=["MaxSupply", "TotalCoinsMined"], index=clustered_df.index)
plot_df["CoinName"] = clustered_df["CoinName"]
plot_df["Class"] = clustered_df["Class"]

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot_df

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
clustered_df[["CoinName","Algorithm","ProofType","MaxSupply","TotalCoinsMined","Class",]].hvplot.table()

In [ ]:
# Print the total number of tradable cryptocurrencies
print(f"There are {clustered_df.shape()} tradable cryptocurrencies."